In [ ]:
!cp drive/MyDrive/НПК/train.zip train.zip
!unzip train.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: train/artificial/1703.png  
  inflating: train/artificial/1065.png  
  inflating: train/artificial/artificial_97.jpg  
  inflating: train/artificial/artificial_700.jpg  
  inflating: train/artificial/artificial_714.jpg  
  inflating: train/artificial/1059.png  
  inflating: train/artificial/artificial_83.jpg  
  inflating: train/artificial/artificial_68.jpg  
  inflating: train/artificial/526.png  
  inflating: train/artificial/240.png  
  inflating: train/artificial/artificial_927.jpg  
  inflating: train/artificial/artificial_1428.jpg  
  inflating: train/artificial/254.png  
  inflating: train/artificial/artificial_933.jpg  
  inflating: train/artificial/532.png  
  inflating: train/artificial/artificial_54.jpg  
  inflating: train/artificial/artificial_1366.jpg  
  inflating: train/artificial/artificial_1400.jpg  
  inflating: train/artificial/268.png  
  inflating: train/artificial/artificial_1414.jpg

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

In [ ]:
batch_size = 8
img_height = 512
img_width = 512

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
    'train',
    validation_split=0.25,
    seed=123,
    subset='training',
    image_size=(img_height, img_width),
    batch_size=batch_size
)

val_ds = tf.keras.utils.image_dataset_from_directory(
    'train',
    validation_split=0.25,
    seed=123,
    subset='validation',
    image_size=(img_height, img_width),
    batch_size=batch_size
)

print(train_ds.class_names)
print(val_ds.class_names)

Found 7236 files belonging to 2 classes.
Using 5427 files for training.
Found 7236 files belonging to 2 classes.
Using 1809 files for validation.
['artificial', 'human']
['artificial', 'human']


In [ ]:
model_input = keras.Input(shape=(None, None, 3), name='img')
x = layers.Resizing(512, 512, input_shape=(512, 512, 3), name='resize')(model_input)
x = layers.Rescaling(1./255, input_shape=(512, 512, 3), name='rescaling')(x)
x = layers.GaussianNoise(0.01, input_shape=(512, 512, 3), name='noise')(x)
x = layers.Conv2D(16, 3, padding='same', activation='relu', input_shape=(512, 512, 3), kernel_regularizer=tf.keras.regularizers.l2(0.0001), name='conv1')(x)
x = layers.MaxPooling2D(name='max_pool1')(x)
x = layers.Conv2D(32, 3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001), name='conv2')(x)
x = layers.MaxPooling2D(name='max_pool2')(x)
x = layers.Conv2D(64, 3, padding='same', activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.0001), name='conv3')(x)
x = layers.MaxPooling2D(name='max_pool3')(x)
x = layers.Flatten(name='flatten')(x)
x = layers.Dense(128, activation='relu', name='dense1', kernel_regularizer=tf.keras.regularizers.l2(0.0001))(x)
x = layers.Dropout(0.2, name='dropout')(x)
model_output = layers.Dense(2, name='dense2')(x)

model = keras.Model(model_input, model_output, name='ai_image_classification')

lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    0.001,
    decay_steps=65,
    decay_rate=0.96,
    staircase=True)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
epochs=10
history = model.fit(
  train_ds,
  validation_data=val_ds,
  steps_per_epoch=65,
  epochs=epochs
)

Epoch 1/10
65/65 [==============================] - 427s 7s/step - loss: 1.2853 - accuracy: 0.5750 - val_loss: 0.7038 - val_accuracy: 0.6683
Epoch 2/10
65/65 [==============================] - 421s 7s/step - loss: 0.7018 - accuracy: 0.6442 - val_loss: 0.6322 - val_accuracy: 0.6871
Epoch 3/10
65/65 [==============================] - 428s 7s/step - loss: 0.6948 - accuracy: 0.6423 - val_loss: 0.6924 - val_accuracy: 0.6915
Epoch 4/10
65/65 [==============================] - 480s 7s/step - loss: 0.6741 - accuracy: 0.6635 - val_loss: 0.6825 - val_accuracy: 0.6009
Epoch 5/10
65/65 [==============================] - 416s 6s/step - loss: 0.6716 - accuracy: 0.6596 - val_loss: 0.6215 - val_accuracy: 0.7098
Epoch 6/10
65/65 [==============================] - 406s 6s/step - loss: 0.6375 - accuracy: 0.7000 - val_loss: 0.6345 - val_accuracy: 0.6479
Epoch 7/10
65/65 [==============================] - 415s 6s/step - loss: 0.6381 - accuracy: 0.7058 - val_loss: 0.6039 - val_accuracy: 0.7043
Epoch 8/10
65